In [ ]:
import paddle
import numpy as np

def load_point_cloud(path):
    tensor = paddle.load(path)
    return tensor.numpy()  # Convert to NumPy array for Open3D


In [ ]:
import open3d as o3d

def visualize_point_cloud(points):
    pcd = o3d.geometry.PointCloud()
    pcd.points = o3d.utility.Vector3dVector(points)  # expects Nx3 array
    o3d.visualization.draw_geometries([pcd])


In [ ]:
import numpy as np
import matplotlib.pyplot as plt

def generate_slices(points, axis='x', num_slices=100, project_to_2d=True):
    """
    points: (N,3) array
    axis: 'x','y', or 'z' — the direction along which to slice
    num_slices: how many bins/slices to create
    project_to_2d: if True, each slice is Nx2 (drops the slicing axis)
    
    Returns:
      slices: list of length num_slices, each an (Mi,2) or (Mi,3) array
      edges:   the (num_slices+1,) bin edges along the chosen axis
    """
    axis_map = {'x':0, 'y':1, 'z':2}
    ax = axis_map[axis]
    
    # 1. find min/max along axis
    coords = points[:, ax]
    min_val, max_val = coords.min(), coords.max()
    
    # 2. create equally spaced edges
    edges = np.linspace(min_val, max_val, num_slices+1)
    
    slices = []
    for i in range(num_slices):
        low, high = edges[i], edges[i+1]
        mask = (coords >= low) & (coords < high)
        sl = points[mask]
        if project_to_2d:
            sl = np.delete(sl, ax, axis=1)  # drop the slicing axis
        slices.append(sl)
    
    return slices, edges


In [ ]:
def display_slices_same_scale(slices, edges, axis='x', n_cols=5, figsize=(15,3)):
    # --- 1. gather global limits -------------------------
    all_xy = np.vstack([sl for sl in slices if sl.size])
    xmin, xmax = all_xy[:,0].min(), all_xy[:,0].max()
    ymin, ymax = all_xy[:,1].min(), all_xy[:,1].max()

    # add a tiny margin so dots at the edge aren’t clipped
    pad_x = 0.02 * (xmax - xmin)
    pad_y = 0.02 * (ymax - ymin)
    xmin, xmax = xmin - pad_x, xmax + pad_x
    ymin, ymax = ymin - pad_y, ymax + pad_y
    # ------------------------------------------------------

    n = len(slices)
    n_rows = int(np.ceil(n / n_cols))
    fig, axes = plt.subplots(n_rows, n_cols, figsize=(figsize[0], figsize[1]*n_rows))

    for idx, sl in enumerate(slices):
        ax = axes.flat[idx]
        if sl.size:
            ax.scatter(sl[:,0], sl[:,1], s=1, c='k')
        else:
            ax.text(0.5, 0.5, 'Empty', ha='center', va='center')
        ax.set_xlim(xmin, xmax)
        ax.set_ylim(ymin, ymax)
        ax.set_aspect('equal', adjustable='box')
        ax.axis('off')
        ax.set_title(f"{axis} ∈ [{edges[idx]:.2f}, {edges[idx+1]:.2f}) \n slice : {idx}")

    for j in range(idx+1, n_rows*n_cols):
        axes.flat[j].axis('off')

    plt.tight_layout()
    plt.show()



In [ ]:
# load your points 
points = load_point_cloud("../Data/PointClouds/DrivAer_F_D_WM_WW_0420.paddle_tensor")

# generate 80 slices along X, projecting each to YZ
slices, edges = generate_slices(points, axis='z', num_slices=50, project_to_2d=True)

# visualize them in a 5-column grid
display_slices_same_scale(slices, edges=edges, axis='z', n_cols=5)


In [ ]:
visualize_point_cloud(points)